In [26]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

You should consider upgrading via the 'c:\users\janbinkowski\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [27]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [28]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [29]:
# Path to data for train
DATA_PATH = os.path.join('C://Users//JanBinkowski//Desktop//SignLanguageRecognition//MP_DATA')

# Actions to detect
actions = np.array(['a', 'b', 'c'])

# Thirty videos worth of data
no_sequences = 100

# frames for one sequence
sequence_length = 30

In [30]:
sequences, labels = [], []

In [31]:
label_map = {label:num for num, label in enumerate(actions)}

In [32]:
label_map

{'a': 0, 'b': 1, 'c': 2}

In [33]:
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [34]:
X = np.array(sequences)

In [35]:
X_sliced_for_training = np.concatenate((X[:80],X[100:180],X[200:280]), axis=0)

In [36]:
X_sliced_for_validation = np.concatenate((X[80:100],X[180:200],X[280:300]), axis=0)

In [37]:
# X_sliced_for_testing = np.concatenate((X[90:100],X[190:200],X[290:300]), axis=0)

In [38]:
y = to_categorical(labels).astype(int)

In [39]:
y_sliced_for_training = np.concatenate((y[:80],y[100:180],y[200:280]), axis=0)

In [40]:
y_sliced_for_validation = np.concatenate((y[80:100],y[180:200],y[280:300]), axis=0)

In [41]:
# y_sliced_for_testing = np.concatenate((y[90:100],y[190:200],y[290:300]), axis=0)

In [42]:
# X_train, X_test, y_train, y_test = train_test_split(X_sliced_for_training, y_sliced_for_training, test_size=0.05)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [44]:
log_dir = os.path.join('C://Users//JanBinkowski//Desktop//SignLanguageRecognition//Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [45]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [46]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [47]:
# model.fit(X_train, y_train, epochs=200, batch_size=4, validation_data=(X_sliced_for_validation, y_sliced_for_validation), callbacks=[tb_callback])
model.fit(X_train, y_train, epochs=200, validation_split=0.2, callbacks=[tb_callback])


Epoch 1/200
8/8 [==============================] - 5s 222ms/step - loss: 1.0956 - categorical_accuracy: 0.3900 - val_loss: 1.0777 - val_categorical_accuracy: 0.9123
Epoch 2/200
8/8 [==============================] - 0s 45ms/step - loss: 1.0673 - categorical_accuracy: 0.7487 - val_loss: 1.0024 - val_categorical_accuracy: 0.3684
Epoch 3/200
8/8 [==============================] - 0s 43ms/step - loss: 1.1475 - categorical_accuracy: 0.6106 - val_loss: 0.8741 - val_categorical_accuracy: 0.5439
Epoch 4/200
8/8 [==============================] - 0s 45ms/step - loss: 0.9117 - categorical_accuracy: 0.5434 - val_loss: 0.9186 - val_categorical_accuracy: 0.5789
Epoch 5/200
8/8 [==============================] - 0s 44ms/step - loss: 0.8937 - categorical_accuracy: 0.6534 - val_loss: 0.8067 - val_categorical_accuracy: 0.6667
Epoch 6/200
8/8 [==============================] - 0s 48ms/step - loss: 0.8031 - categorical_accuracy: 0.6880 - val_loss: 0.7225 - val_categorical_accuracy: 0.8772
Epoch 7/200
8/8

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 64)            48896     
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 99        
Total params: 203,459
Trainable params: 203,459
Non-trainable params: 0
________________________________________________

In [49]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=1)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 30 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions.shape)

Evaluate on test data
15/15 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - categorical_accuracy: 1.0000
test loss, test acc: [0.0, 1.0]
Generate predictions for 30 samples
predictions shape: (15, 3)


In [50]:
model.save('C://Users//JanBinkowski//Desktop//SignLanguageRecognition//Weights//action.h5')